In [ ]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import KFold
from scipy.stats import sem, t
import numpy as np
import copy

class MineralDataset(Dataset):
    def __init__(self, img_dir, dataframe, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        file_name = row['Segmented Filename']
        label = row['Class']
        img_path = os.path.join(self.img_dir, file_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

IMAGE_DIR = '/Mineral/CPX seg'
TRAIN_SPLIT_FILE = '/Mineral/train_split.csv'
VAL_SPLIT_FILE = '/Mineral/val_split.csv'
train_data = pd.read_csv(TRAIN_SPLIT_FILE)
val_data = pd.read_csv(VAL_SPLIT_FILE)
#Complete training set
data = pd.concat([train_data, val_data], ignore_index=True)

dataset = MineralDataset(
    img_dir=IMAGE_DIR,
    dataframe=data,
    transform=None 
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def initialize_model(num_classes=3):
    model = models.resnet34(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    model = model.to(device)
    return model

def train_and_evaluate_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, device='cuda'):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_corrects = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_corrects += torch.sum(preds == labels.data).item()

        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_corrects / len(train_loader.dataset)
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)

        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data).item()

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects / len(val_loader.dataset)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f'Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    return model, history

# K-fold training, set 100 epochs to allow model convergent on each fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
num_epochs = 100

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f"Fold {fold+1}")

    
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

 
    train_subset.dataset.transform = transform_train
    val_subset.dataset.transform = transform_val

  
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

    labels = [dataset[idx][1] for idx in train_idx]
    classes = np.unique(labels)
    class_weights = compute_class_weight('balanced', classes=classes, y=labels)
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

    model = initialize_model(num_classes=3)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    optimizer = optim.Adam(model.fc.parameters(), lr=0.0001)

    trained_model, history = train_and_evaluate_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs, device=device)


    torch.save(trained_model, f'/Mineral/complete_model_fold_{fold+1}.pth')

    print(f"Fold {fold+1} - Train Loss: {history['train_loss'][-1]:.4f}, Train Accuracy: {history['train_acc'][-1]:.2f}%, Val Loss: {history['val_loss'][-1]:.4f}, Val Accuracy: {history['val_acc'][-1]:.2f}%")


Fold 1


C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/100: Train Loss: 1.2235, Train Acc: 0.2515, Val Loss: 1.1629, Val Acc: 0.3193
Epoch 2/100: Train Loss: 1.1353, Train Acc: 0.3678, Val Loss: 1.0964, Val Acc: 0.4066
Epoch 3/100: Train Loss: 1.0598, Train Acc: 0.4441, Val Loss: 1.0379, Val Acc: 0.4940
Epoch 4/100: Train Loss: 1.0090, Train Acc: 0.5363, Val Loss: 0.9779, Val Acc: 0.6235
Epoch 5/100: Train Loss: 0.9674, Train Acc: 0.5665, Val Loss: 0.9457, Val Acc: 0.6024
Epoch 6/100: Train Loss: 0.9331, Train Acc: 0.6329, Val Loss: 0.9146, Val Acc: 0.6205
Epoch 7/100: Train Loss: 0.9014, Train Acc: 0.6556, Val Loss: 0.8932, Val Acc: 0.6386
Epoch 8/100: Train Loss: 0.8664, Train Acc: 0.6881, Val Loss: 0.8637, Val Acc: 0.6988
Epoch 9/100: Train Loss: 0.8396, Train Acc: 0.6911, Val Loss: 0.8484, Val Acc: 0.7078
Epoch 10/100: Train Loss: 0.8235, Train Acc: 0.7054, Val Loss: 0.8296, Val Acc: 0.7078
Epoch 11/100: Train Loss: 0.8001, Train Acc: 0.7122, Val Loss: 0.8186, Val Acc: 0.6807
Epoch 12/100: Train Loss: 0.7776, Train Acc: 0.7243,

C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/100: Train Loss: 1.1211, Train Acc: 0.3449, Val Loss: 1.1034, Val Acc: 0.3353
Epoch 2/100: Train Loss: 1.0360, Train Acc: 0.5094, Val Loss: 1.0091, Val Acc: 0.4713
Epoch 3/100: Train Loss: 0.9855, Train Acc: 0.5796, Val Loss: 0.9745, Val Acc: 0.6435
Epoch 4/100: Train Loss: 0.9385, Train Acc: 0.5864, Val Loss: 0.9327, Val Acc: 0.6526
Epoch 5/100: Train Loss: 0.8940, Train Acc: 0.7026, Val Loss: 0.8999, Val Acc: 0.6858
Epoch 6/100: Train Loss: 0.8746, Train Acc: 0.6770, Val Loss: 0.8743, Val Acc: 0.6888
Epoch 7/100: Train Loss: 0.8469, Train Acc: 0.7336, Val Loss: 0.8487, Val Acc: 0.6828
Epoch 8/100: Train Loss: 0.8268, Train Acc: 0.6981, Val Loss: 0.8349, Val Acc: 0.6918
Epoch 9/100: Train Loss: 0.8090, Train Acc: 0.7072, Val Loss: 0.8203, Val Acc: 0.7190
Epoch 10/100: Train Loss: 0.7968, Train Acc: 0.7298, Val Loss: 0.8004, Val Acc: 0.7100
Epoch 11/100: Train Loss: 0.7717, Train Acc: 0.7419, Val Loss: 0.7906, Val Acc: 0.7100
Epoch 12/100: Train Loss: 0.7591, Train Acc: 0.7298,

...

Epoch 1/100: Train Loss: 1.1518, Train Acc: 0.3208, Val Loss: 1.1206, Val Acc: 0.3958
Epoch 2/100: Train Loss: 1.0669, Train Acc: 0.4543, Val Loss: 1.0656, Val Acc: 0.5257
Epoch 3/100: Train Loss: 1.0140, Train Acc: 0.5487, Val Loss: 1.0002, Val Acc: 0.6133
Epoch 4/100: Train Loss: 0.9617, Train Acc: 0.5977, Val Loss: 0.9587, Val Acc: 0.6828
Epoch 5/100: Train Loss: 0.9164, Train Acc: 0.6377, Val Loss: 0.9158, Val Acc: 0.7009
Epoch 6/100: Train Loss: 0.8814, Train Acc: 0.6709, Val Loss: 0.8877, Val Acc: 0.7100
Epoch 7/100: Train Loss: 0.8606, Train Acc: 0.7147, Val Loss: 0.8623, Val Acc: 0.7492
Epoch 8/100: Train Loss: 0.8345, Train Acc: 0.6943, Val Loss: 0.8391, Val Acc: 0.7402
Epoch 9/100: Train Loss: 0.8116, Train Acc: 0.7366, Val Loss: 0.8147, Val Acc: 0.7402
Epoch 10/100: Train Loss: 0.7983, Train Acc: 0.7215, Val Loss: 0.8030, Val Acc: 0.7644
Epoch 11/100: Train Loss: 0.7941, Train Acc: 0.7147, Val Loss: 0.7993, Val Acc: 0.7734
Epoch 12/100: Train Loss: 0.7779, Train Acc: 0.7555,

C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/100: Train Loss: 1.0803, Train Acc: 0.4687, Val Loss: 1.0114, Val Acc: 0.5740
Epoch 2/100: Train Loss: 1.0060, Train Acc: 0.5245, Val Loss: 0.9504, Val Acc: 0.6435
Epoch 3/100: Train Loss: 0.9677, Train Acc: 0.5834, Val Loss: 0.9151, Val Acc: 0.7009
Epoch 4/100: Train Loss: 0.9196, Train Acc: 0.6521, Val Loss: 0.8899, Val Acc: 0.6828
Epoch 5/100: Train Loss: 0.8957, Train Acc: 0.6453, Val Loss: 0.8721, Val Acc: 0.7039
Epoch 6/100: Train Loss: 0.8730, Train Acc: 0.7094, Val Loss: 0.8451, Val Acc: 0.7100
Epoch 7/100: Train Loss: 0.8438, Train Acc: 0.6913, Val Loss: 0.8304, Val Acc: 0.7190
Epoch 8/100: Train Loss: 0.8234, Train Acc: 0.7057, Val Loss: 0.8162, Val Acc: 0.7069
Epoch 9/100: Train Loss: 0.8081, Train Acc: 0.7117, Val Loss: 0.8055, Val Acc: 0.7190
Epoch 10/100: Train Loss: 0.7938, Train Acc: 0.7019, Val Loss: 0.7929, Val Acc: 0.7492
Epoch 11/100: Train Loss: 0.7707, Train Acc: 0.7374, Val Loss: 0.7854, Val Acc: 0.7523
Epoch 12/100: Train Loss: 0.7659, Train Acc: 0.7064,

C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/100: Train Loss: 1.2411, Train Acc: 0.3419, Val Loss: 1.1709, Val Acc: 0.2024
Epoch 2/100: Train Loss: 1.0937, Train Acc: 0.4136, Val Loss: 1.0601, Val Acc: 0.4894
Epoch 3/100: Train Loss: 1.0315, Train Acc: 0.5223, Val Loss: 1.0108, Val Acc: 0.5680
Epoch 4/100: Train Loss: 0.9824, Train Acc: 0.5985, Val Loss: 0.9684, Val Acc: 0.6133
Epoch 5/100: Train Loss: 0.9345, Train Acc: 0.6204, Val Loss: 0.9335, Val Acc: 0.6344
Epoch 6/100: Train Loss: 0.9042, Train Acc: 0.6936, Val Loss: 0.9082, Val Acc: 0.6073
Epoch 7/100: Train Loss: 0.8701, Train Acc: 0.6551, Val Loss: 0.8841, Val Acc: 0.6858
Epoch 8/100: Train Loss: 0.8396, Train Acc: 0.7366, Val Loss: 0.8650, Val Acc: 0.6465
Epoch 9/100: Train Loss: 0.8359, Train Acc: 0.6921, Val Loss: 0.8497, Val Acc: 0.7311
Epoch 10/100: Train Loss: 0.8057, Train Acc: 0.7442, Val Loss: 0.8311, Val Acc: 0.6858
Epoch 11/100: Train Loss: 0.7838, Train Acc: 0.7238, Val Loss: 0.8242, Val Acc: 0.7251
Epoch 12/100: Train Loss: 0.7744, Train Acc: 0.7532,

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
import os


class MineralDataset(Dataset):
    def __init__(self, img_dir, dataframe, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        file_name = row['Segmented Filename']
        label = row['Class']
        img_path = os.path.join(self.img_dir, file_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label


transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


IMAGE_DIR = '/Mineral/CPX seg'
TEST_SPLIT_FILE = '/Mineral/test_split.csv'
test_data = pd.read_csv(TEST_SPLIT_FILE)
test_dataset = MineralDataset(img_dir=IMAGE_DIR, dataframe=test_data, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


test_labels = []
for _, labels in test_loader:
    test_labels.extend(labels.numpy())
test_labels = np.array(test_labels)


all_model_predictions = []
for fold in range(1, 6):
    model_path = f'/Mineral/complete_model_fold_{fold}.pth'
    model = torch.load(model_path)
    model.to(device)
    model.eval()

    fold_predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            fold_predictions.append(probabilities.cpu().numpy())

    fold_predictions = np.concatenate(fold_predictions)
    all_model_predictions.append(fold_predictions)

#final prediction label
predicted_classes = np.array([np.argmax(preds, axis=1) for preds in all_model_predictions])
final_predictions = []
for sample_predictions in predicted_classes.T:
    values, counts = np.unique(sample_predictions, return_counts=True)
    most_frequent = values[np.argmax(counts)]
    final_predictions.append(most_frequent)
final_predictions = np.array(final_predictions)



In [ ]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import copy
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, recall_score, precision_score

criterion = nn.CrossEntropyLoss()

def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    all_probs = []
    all_labels = []
    all_losses = []

    with torch.no_grad(): 
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            probabilities = torch.softmax(outputs, dim=1)
            all_probs.append(probabilities.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            all_losses.append(loss.cpu().numpy())

    all_probs = np.concatenate(all_probs)
    all_labels = np.concatenate(all_labels)
    all_losses = np.array(all_losses)
    return all_probs, all_labels, all_losses

def bootstrap_metric(y_true, y_prob, metric_func, n_bootstrap=1000, alpha=0.05, **kwargs):
    rng = np.random.default_rng()
    indices = np.arange(len(y_true))
    bootstrapped_scores = []
    for _ in range(n_bootstrap):
        sample_indices = rng.choice(indices, size=len(indices), replace=True)
        if metric_func == roc_auc_score:
            score = metric_func(y_true[sample_indices], y_prob[sample_indices], **kwargs)
        else:
            y_pred = np.argmax(y_prob[sample_indices], axis=1)
            score = metric_func(y_true[sample_indices], y_pred, **kwargs)
        bootstrapped_scores.append(score)
    sorted_scores = np.sort(bootstrapped_scores)
    ci_lower = sorted_scores[int((alpha/2) * n_bootstrap)]
    ci_upper = sorted_scores[int((1 - alpha/2) * n_bootstrap)]
    return np.mean(bootstrapped_scores), ci_lower, ci_upper

def bootstrap_loss(losses, n_bootstrap=1000, alpha=0.05):
    rng = np.random.default_rng()
    indices = np.arange(len(losses))
    bootstrapped_losses = []
    for _ in range(n_bootstrap):
        sample_indices = rng.choice(indices, size=len(indices), replace=True)
        sample_loss = losses[sample_indices]
        bootstrapped_losses.append(np.mean(sample_loss))
    sorted_losses = np.sort(bootstrapped_losses)
    ci_lower = sorted_losses[int((alpha/2) * n_bootstrap)]
    ci_upper = sorted_losses[int((1 - alpha/2) * n_bootstrap)]
    return np.mean(bootstrapped_losses), ci_lower, ci_upper


test_probs, test_labels, test_losses = evaluate_model(model, test_loader, criterion, device)

class_labels = np.unique(test_labels)
metrics = {}
for class_label in class_labels:
    auc_roc_mean, auc_roc_ci_low, auc_roc_ci_high = bootstrap_metric(test_labels == class_label, test_probs[:, class_label], roc_auc_score)
    balanced_acc_mean, balanced_acc_ci_low, balanced_acc_ci_high = bootstrap_metric(test_labels, test_probs, lambda y_true, final_predictions: balanced_accuracy_score(y_true == class_label, final_predictions == class_label))
    sensitivity_mean, sensitivity_ci_low, sensitivity_ci_high = bootstrap_metric(test_labels, test_probs, lambda y_true, final_predictions: recall_score(y_true == class_label, final_predictions == class_label, zero_division=0))
    specificity_mean, specificity_ci_low, specificity_ci_high = bootstrap_metric(test_labels, test_probs, lambda y_true, final_predictions: precision_score(y_true == class_label, final_predictions == class_label, zero_division=0))

    metrics[class_label] = {
        "AUC-ROC": (auc_roc_mean, auc_roc_ci_low, auc_roc_ci_high),
        "Balanced Accuracy": (balanced_acc_mean, balanced_acc_ci_low, balanced_acc_ci_high),
        "Sensitivity": (sensitivity_mean, sensitivity_ci_low, sensitivity_ci_high),
        "Specificity": (specificity_mean, specificity_ci_low, specificity_ci_high)
    }


print("Class\t\tAUC-ROC\t\t\tBalanced Accuracy\tSensitivity\t\tSpecificity")
for class_label, metric_values in metrics.items():
    auc_roc = metric_values["AUC-ROC"]
    balanced_acc = metric_values["Balanced Accuracy"]
    sensitivity = metric_values["Sensitivity"]
    specificity = metric_values["Specificity"]
    print(f"{class_label}\t\t{auc_roc[0]:.2f} ({auc_roc[1]:.2f}-{auc_roc[2]:.2f})\t"
          f"{balanced_acc[0]:.2f} ({balanced_acc[1]:.2f}-{balanced_acc[2]:.2f})\t"
          f"{sensitivity[0]:.2f} ({sensitivity[1]:.2f}-{sensitivity[2]:.2f})\t"
          f"{specificity[0]:.2f} ({specificity[1]:.2f}-{specificity[2]:.2f})")


auc_roc_mean, auc_roc_ci_low, auc_roc_ci_high = bootstrap_metric(test_labels, test_probs, roc_auc_score, multi_class='ovr')
balanced_acc_mean, balanced_acc_ci_low, balanced_acc_ci_high = bootstrap_metric(test_labels, test_probs, balanced_accuracy_score)
sensitivity_mean, sensitivity_ci_low, sensitivity_ci_high = bootstrap_metric(test_labels, test_probs, lambda y_true, final_predictions: recall_score(y_true, final_predictions, average='macro', zero_division=0))
specificity_mean, specificity_ci_low, specificity_ci_high = bootstrap_metric(test_labels, test_probs, lambda y_true, final_predictions: precision_score(y_true, final_predictions, average='macro', zero_division=0))


loss_mean, loss_ci_low, loss_ci_high = bootstrap_loss(test_losses)

print(f"Overall\t\t{auc_roc_mean:.2f} ({auc_roc_ci_low:.2f}-{auc_roc_ci_high:.2f})\t"
      f"{balanced_acc_mean:.2f} ({balanced_acc_ci_low:.2f}-{balanced_acc_ci_high:.2f})\t"
      f"{sensitivity_mean:.2f} ({sensitivity_ci_low:.2f}-{sensitivity_ci_high:.2f})\t"
      f"{specificity_mean:.2f} ({specificity_ci_low:.2f}-{specificity_ci_high:.2f})")
print(f"Loss\t\t{loss_mean:.4f} ({loss_ci_low:.4f}-{loss_ci_high:.4f})")

Class		AUC-ROC			Balanced Accuracy	Sensitivity		Specificity
0		0.95 (0.93-0.98)	0.89 (0.86-0.93)	0.91 (0.85-0.96)	0.79 (0.71-0.86)
1		0.90 (0.85-0.94)	0.83 (0.79-0.87)	0.77 (0.70-0.83)	0.90 (0.86-0.95)
2		0.73 (0.62-0.83)	0.67 (0.58-0.77)	0.45 (0.26-0.65)	0.32 (0.17-0.48)
Overall		0.86 (0.81-0.90)	0.71 (0.64-0.78)	0.71 (0.64-0.78)	0.67 (0.61-0.73)
Loss		0.6069 (0.5260-0.6880)
